# **STAT 587 &mdash; Data Science I** &mdash; Homework 2
**Winter 2026**

## **Imports & Setup**

In [1]:
import sys
from pathlib import Path

# Add project root to Python path
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [2]:
import numpy as np
import random

# Set Seed 
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

In [3]:
from src.paths import ensure_dirs, DATA_DIR, FIG_DIR, TAB_DIR

ensure_dirs()

## **Question 1:** College Data (ISLP)


### **(a)** Linear Regression

### **(b)** Regression Tree

### **(c)** Pruning via Cross-Validation

### **(d)** Bagging

### **(e)** Random Forest

### **(f)** Comparison

## **Question 2:** Business School Admissions

### **(a)** Exploratory Analysis

### **(b)** LDA


### **(c)** QDA

### **(d)** KNN

### **(e)** Comparison